In [9]:
import sympy as sp
import numpy as np 
from numpy import radians
from time import perf_counter
import casadi as ca

In [10]:
num_poles = 2

pole_as = [ca.SX.sym(f'a_{i+1}') for i in range(num_poles)]
pole_ls = [ca.SX.sym(f'l_{i+1}') for i in range(num_poles)]
pole_ms = [ca.SX.sym(f'm_{i+1}') for i in range(num_poles)]
pole_ds = [ca.SX.sym(f'd_{i+1}') for i in range(num_poles)]
pole_Js = [ca.SX.sym(f'J_{i+1}') for i in range(num_poles)]

m_c = ca.SX.sym('m_c')
g = ca.SX.sym('g')

In [17]:
t = ca.MX.sym("t")
s = ca.MX.sym("s")

In [ ]:
t = ca.SX.sym("t")
s = ca.Function(

d_s = sp.diff(s, t)
dd_s = sp.diff(d_s, t)
thetas = [sp.Function(f"\\theta_{i+1}")(t) for i in range(num_poles)]
d_thetas = [sp.diff(theta,t) for theta in thetas]
dd_thetas = [sp.diff(d_theta,t) for d_theta in d_thetas]
u = dd_s
torque = sp.Symbol("\\tau")

In [ ]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])
print(pole_pcs)

[[-a_1*sin(\theta_1(t)) + s(t), a_1*cos(\theta_1(t))], [-a_2*sin(\theta_2(t)) - l_1*sin(\theta_1(t)) + s(t), a_2*cos(\theta_2(t)) + l_1*cos(\theta_1(t))]]


In [ ]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T = [1/2*m_c*d_s**2]
for m, pc, J, d_theta in zip(pole_ms, pole_pcs, pole_Js, d_thetas):
    d_pc = [sp.diff(p,t) for p in pc]
    T_p = 1/2*m*square_sum(d_pc) + 1/2*J*d_theta**2
    T.append(T_p)
    
V = 0   
for m, pc in zip(pole_ms, pole_pcs):
    V += g*m*pc[1]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

In [ ]:
L = T[0]-V
eqs = []
lh = sp.diff(sp.diff(L, d_s),t)
lh -= sp.diff(L, s) + sp.diff(R, d_s)
rh = torque
eqs.append(lh-rh)
for theta, d_theta, T_p in zip(thetas, d_thetas, T[1:]):
    L = T_p-V
    lh = sp.diff(sp.diff(L, d_theta),t)
    lh -= sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)

In [7]:
sols = sp.solve(eqs, dd_thetas + [torque])
for key, value in sols.items():
    sols[key] = sp.simplify(value)
    display(sols[key])

NameError: name 'eqs' is not defined

In [21]:
values = np.array([10, 0, radians(0), 0, radians(0), 0, radians(0), 0] + [0])
variables = [s, d_s]
for theta, d_theta in zip(thetas, d_thetas):
    variables.append(theta)
    variables.append(d_theta)
variables.append(u)

start_time = perf_counter()
# substitute the values of the variables into the solutions
sol_sub = sols.copy()
for k, v in sols.items():
    a = v.subs({var: value for var, value in zip(variables, values)})
    print(a)
print(f"Time: {perf_counter()-start_time}")

0
Time: 0.0021551999961957335
